In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from bs4 import BeautifulSoup
import requests
import json
from IPython.display import Markdown
from IPython.display import update_display


In [3]:
load_dotenv(override="true")
api_key = os.getenv("OPENAI_API_KEY")
if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

API key looks good so far


In [4]:
openai = OpenAI()
MODAL='gpt-5-nano'

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

def fetch_website_contents(url):
    """
    Return the title and contents of the website at the given url;
    truncate to 2,000 characters as a sensible limit
    """
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    title = soup.title.string if soup.title else "No title found"
    if soup.body:
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        text = soup.body.get_text(separator="\n", strip=True)
    else:
        text = ""
    return (title + "\n\n" + text)[:2_000]


In [6]:
link_system_prompt = """
You are a UX designer.
You are given a website and you look at it and find ways to improve how the UX could look better.
You should look at options like making the website more modern and catchy.
It could br usage of better css or animations or the way different components are rendered
"""

In [9]:
def get_user_prompt(url):
    user_prompt = f"""
    here is a link to the website {url} -
    Please provide options to make this website better
     """
    return user_prompt

In [10]:
print(get_user_prompt('https://secondary.tzero.com/investors/register'))


    here is a link to the website https://secondary.tzero.com/investors/register -
    Please provide options to make this website better
     


In [11]:
def suggest_better_design_options(url):
    response = openai.chat.completions.create(
        model=MODAL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": 'user', "content": get_user_prompt(url)}
        ]
    )
    result = response.choices[0].message.content
    return result

In [13]:
suggest_better_design_options('https://secondary.tzero.com/investors/register')

'Here are UX-driven options to improve the registration experience for investors on that page. I’ve split them into quick wins, and three design directions you can choose from (or mix). Each option includes concrete changes, rationale, and how to implement them.\n\n Quick wins (high impact, low effort)\n- Make it a single-column, mobile-friendly layout that scales down gracefully. Remove long side-by-side sections; ensure readability on phones.\n- Use real-time inline validation. As users fill fields, show green checkmarks for valid input or clear error messages under the field (e.g., “Email is not valid” or “Phone must include country code”).\n- Implement a 3-step progressive form rather than one long page. Show a progress indicator (Step 1 of 3) and only reveal the next section after the current one is valid.\n- Add field masking and autofill optimizations. Password strength meter, phone number masking, and automatic date/ZIP formats improve speed and accuracy.\n- Save progress local

In [14]:
def make_better_UX(url):
    contents = suggest_better_design_options(url)
    result = f"## Suggested Changes:\n\n{contents}\n##:\n"
    return result

In [15]:
print(make_better_UX("https://secondary.tzero.com/investors/register"))

## Suggested Changes:

I can’t load the page directly from here, but I can give you a comprehensive set of UX improvement options specifically for an investor registration flow like this. They’re organized so you can pick quick wins vs. longer-term redesign work.

Top priorities (quick wins you can implement in 1–2 sprints)
- Multi-step registration flow with a progress indicator
  - Break the form into 4–6 logical steps (e.g., Personal details, Contact & location, Investment experience, Compliance & disclosures, Review & submit).
  - Show a clear progress bar and the ability to save progress and resume later.
- Clear value proposition above the fold
  - A concise line or two explaining benefits of registering (trust, speed, access to exclusive investor information) and a mini “why register” bullets.
- Accessible, labeled form fields with real-time validation
  - Always show visible labels, not only placeholders.
  - Inline validation (green checkmarks for valid; red for errors) as use

In [25]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

In [30]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
    You are looking at a company called: {company_name}
    Here are the contents of its landing page and other relevant pages;
    use this information to build a short brochure of the company in markdown without code blocks.\n\n"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [31]:
get_brochure_user_prompt("tzero", 'https://www.tzero.com')

'\n    You are looking at a company called: tzero\n    Here are the contents of its landing page and other relevant pages;\n    use this information to build a short brochure of the company in markdown without code blocks.\n\n## Landing Page:\n\ntZERO.com\n\nRegister\nSign In\nTokenize\nTokenize\nCapital Formation\nIssuance Infrastructure\nSecondary Trading\nTrade\nTrade\nInvesting with tZERO\nExplore Offerings\nLiquidity\nCustody & Security\nConnect\nConnect\nGlobal Rails\nAPI-suite\nIntegrations\nContent Hub\nContent Hub\nNews\nMedia\nInsights\nPodcast\nLearn\nCompany\nCompany\nPartners\nTeam\nCareers\nContact\nLegal Hub\nWall\nStreet,\nTokenized\nRedefining\nHow\nCapital\nis\nRaised,\nTraded,\nand\nOwned\nTokenize\nBring every asset on-chain.\nTurn real-world assets into digital opportunity\nLearn more\nTrade\nAccess tokenized assets.\nTransact with transparency, speed, and trust\nLearn more\nConnect\nBuild beyond borders\nJoin a network that bridges traditional finance and blockcha

In [33]:

def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": 'system', 'content': brochure_system_prompt},
            {'role': 'user', 'content': get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [34]:
create_brochure("tzero", 'https://www.tzero.com')

# tZERO Brochure

---

## About tZERO

**Where Wall Street Meets Web3**

tZERO is a pioneering company redefining how capital is raised, traded, and owned by bridging traditional finance and Web3 technologies. By building regulated digital asset infrastructure, tZERO unites blockchain efficiency with institutional-grade compliance to create a seamless ecosystem for tokenized finance.

Tokenization reshapes markets by giving assets a shared digital language, enabling them to trade, settle, and interact seamlessly across platforms, asset classes, and borders. tZERO transforms traditional assets—including public and private securities, funds, derivatives, and real-world assets like real estate and art—into on-chain tokens, unlocking a new era of financial transparency, efficiency, and interoperability.

---

## What We Do

**Tokenize | Trade | Connect**

- **Tokenize:**  
  Enable public and private companies, banks, broker-dealers, and asset managers to digitize equity, debt, funds, and real-world assets through regulated, compliant token issuance. Expand product offerings via digital securities and launch tokenized funds and yield products accessible to global investors.

- **Trade:**  
  Provide a marketplace for secondary trading of tokenized assets, delivering transparency, speed, trust, and liquidity. Our platform supports smooth transaction settlement with institutional-grade security.

- **Connect:**  
  Facilitate a global network that bridges traditional finance and blockchain with interoperable digital rails, APIs, and cross-border access. This connectivity fosters collaboration among issuers, investors, broker-dealers, financial institutions, and fintech partners worldwide.

---

## Our Vision

To create a multi-asset digital ecosystem that enables:

- Cross-asset interoperability — seamless trading across equities, debt, funds, and digital assets.
- Global 24/7 market convergence — enabling continuous, cross-border access via smart contract-driven infrastructure.
- Efficiency, transparency, and customization — removing intermediaries, reducing settlement times, and enhancing operational control.

---

## Industry-Leading Infrastructure

tZERO’s end-to-end platform provides:

- Regulated frameworks combining tokenization, trading, settlement, and custody.
- Institutional-grade security protocols safeguarding assets with transparency.
- Scalable, blockchain-powered technology enabling new product innovation and capital formation.

---

## Who We Serve

Our ecosystem supports the entire lifecycle of tokenized finance. We empower:

- **Issuers:** companies, banks, and asset managers digitizing securities and real-world assets.
- **Investors:** accessing diversified, tokenized offerings with liquidity options.
- **Financial Institutions and Partners:** leveraging our APIs and global rails for seamless integration and participation.

---

## Leadership Team

Our Board of Directors includes experts in global financial markets, technology, and digital assets. The leadership drives innovation at the crossroads of regulated finance and blockchain technology, ensuring strategic vision and trusted execution.

---

## Careers at tZERO

We seek talented professionals passionate about transforming capital markets through blockchain innovation. Joining tZERO means:

- Working in a cutting-edge, fast-paced environment at the forefront of digital finance.
- Collaborating with industry leaders dedicated to transparency, compliance, and global market connectivity.
- Opportunities to contribute to pioneering solutions with real-world impact.

Explore career opportunities on our website and help shape the future of finance.

---

## Connect With Us

Discover how tZERO can help you tokenize assets, trade with confidence, and connect internationally in a compliant, innovative ecosystem.

Visit: [tZERO.com](https://tZERO.com)  
Contact us to learn more about partnering, investing, or working together.

---

*Redefining How Capital is Raised, Traded, and Owned — tZERO: Your Partner in a Web3 World.*

In [36]:
def stream_brochure_content(company_name, url):
    stream = openai.chat.completions.create(
        model='gpt-4.1-mini',
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
          stream=True
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [37]:
stream_brochure_content("tzero", 'https://www.tzero.com')

# tZERO Brochure

## About tZERO  
tZERO is redefining how capital is raised, traded, and owned by bridging the gap between traditional finance and the evolving Web3 digital economy. By combining blockchain efficiency with institutional-grade compliance, tZERO builds regulated digital asset infrastructure that powers a multi-asset ecosystem focused on transparency, interoperability, and scalability.

## Our Vision  
To reshape global capital markets by bringing every asset on-chain. We enable tokenization that allows public and private securities, funds, derivatives, and real-world assets (such as real estate and art) to be digitized for seamless on-chain trading, settlement, and interaction across platforms and borders.

## What We Offer  
### Tokenize  
- Transform traditional assets into digital tokens through regulated, compliant issuance infrastructure.
- Tokenize equity, debt, funds, and real-world assets for issuers including banks, broker-dealers, and asset managers.
- Launch tokenized funds and yield products accessible globally.

### Trade  
- Access a marketplace for tokenized assets, allowing investors to transact with speed, transparency, and trust.
- Support for secondary trading with swift settlement, custody, and security integrated end-to-end.

### Connect  
- Build beyond borders by joining a network that bridges traditional capital markets and blockchain technology.
- Utilize global rails and an API suite that enable interoperability and continuous cross-border trading 24/7.

## Why tZERO?  
- **End-to-End Infrastructure:** Tokenization, trading, settlement, and custody unified into a seamless, regulated ecosystem.  
- **Industry-Leading Security:** Institutional-grade controls ensuring trust and compliance in digital asset markets.  
- **Cross-Asset Interoperability:** Trade equities, debt, funds, and digital assets seamlessly on a single platform.  
- **Global Access:** A smart-contract-driven infrastructure enabling round-the-clock, cross-border access to markets.  
- **Reduced Friction:** Minimized intermediaries and faster settlement times improve operational efficiency.

## Our Customers  
tZERO serves a broad ecosystem including:  
- Issuers (public/private companies, banks, asset managers) looking to digitize and innovate their capital formation.  
- Investors seeking transparent and efficient access to tokenized financial products.  
- Broker-dealers and financial institutions requiring compliant and interoperable trading infrastructure.  
- Fintech partners integrating tokenized finance solutions into their offerings.

## Company Culture & Careers  
At tZERO, innovation meets compliance. We foster a culture that values transparency, collaboration, and pioneering new standards for global finance in the digital age. Our team comprises experts in global markets, blockchain technology, and digital assets dedicated to ushering in the future of capital markets.

**Careers:**  
Join us if you are passionate about transforming finance through technology and want to be part of the bridge connecting Wall Street with Web3. Explore roles across engineering, compliance, business development, and product management.

## Leadership  
Our board of directors includes seasoned leaders with deep expertise in global capital markets and blockchain innovation, guiding tZERO’s vision to integrate institutional investment rigor with the flexibility and dynamism of tokenized markets.

---

**Contact & Learn More**  
Visit [tZERO.com](https://www.tzero.com) to explore our offerings, partner with us, or join the team revolutionizing the future of finance.  
Experience how Wall Street meets Web3 with tZERO—bringing every asset on-chain, today.